In [1]:
import pandas as pd

import nltk
# nltk.download('stopwords')
# nltk.download('punkt_tab')

from nltk.corpus import stopwords
from gensim.corpora import Dictionary
from gensim.models import TfidfModel

from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

import json


# Data Modeling 
# pick two topics to compare
# develop a way to identify which topic a post is talking about 
# check occurences of different topics of conversation over time
#   could these different conversation topics appear during a game period more often?


In [2]:

def text2tokens(text):
	stop_words = set(stopwords.words('english'))
	ps = PorterStemmer()
	text = text.lower()
	textList = word_tokenize(text)
	textList = [word for word in textList if word not in stop_words and len(word) >= 3]

	textList = [ps.stem(word) for word in textList]
	return textList
    

def gen_bow(df, column):
	df['tokens'] = df[column].apply(text2tokens)
	dct = Dictionary(df['tokens'])
	dct.filter_extremes(no_below=5, no_above=0.5)
	df['bow'] = df['tokens'].apply(dct.doc2bow)
	worddict = {}

	for i in range(len(df)):
		for token in df.at[i,'bow']:
			if token in list(worddict.keys()):
				worddict[token] += 1
			else: 
				worddict[token] = 1

	df.drop('tokens', axis=1, inplace=True)	
	return df, worddict

def get_post(jobj):
	data_obj = jobj[0]['data']['children'][0]['data']
	txt = data_obj['selftext']
	author = data_obj['author']
	time = data_obj['created_utc']
	
	return {
		'author': author,
		'created_utc': time,
		'text': txt
	}

def get_comment(jobj):
	comments = []
	for comment in jobj[1]['data']['children']:
		if 'author' in comment['data']\
		and 'created_utc' in comment['data']\
		and 'body' in comment['data']:
			cobj = {
				'author': comment['data']['author'],
				'created_utc': comment['data']['created_utc'],
				'text': comment['data']['body']
			}
			comments.append(cobj)

	return comments

# print(text2tokens("In this offering, one only has to view the current Westminster Kennel Club's annual offering to see the parallels."))



In [ ]:
or_posts = []

with open('orioles.json', 'r') as f:
	lines = f.readlines()

for line in lines:
	data_obj = json.loads(line)
	dobj = get_post(data_obj)
	or_posts.append(dobj)
	comments = get_comment(data_obj)
	or_posts.extend(comments)

or_df = pd.DataFrame(or_posts)
or_df


,author,created_utc,text
0,StrategyGameventures,1.620252e+09,first career complete game no-hitter for Means!
1,mistermachiano,1.620253e+09,#FIRST NO HITTER IN 30 YEARS LETS GOOOOOOOO
2,TheIronBug,1.620253e+09,I can't believe the son of a bitch actually di...
3,Greyshot26,1.620253e+09,EVERYBODY GET THE FUCK IN HERE PAT HIT HIS FIR...
4,jshokie1,1.620252e+09,WE'VE SUCKED FOR SO LONG HOW DO I TAKE THIS IN...
...,...,...,...
609,Dartmouthest,1.738018e+09,Hell yeah! Love it. Also Canada banned news on...
610,snoogans675,1.738025e+09,"I love the Os, cold beer, and not supporting t..."
611,lionheart4life,1.738028e+09,"Good, they are all trash."
612,27thStreet,1.738032e+09,Thank you.


In [37]:
ny_posts = []

with open('nyyankees.json', 'r') as f:
	lines = f.readlines()

for line in lines:
	data_obj = json.loads(line)
	dobj = get_post(data_obj)
	ny_posts.append(dobj)
	comments = get_comment(data_obj)
	ny_posts.extend(comments)

ny_df = pd.DataFrame(ny_posts)
ny_df


,author,created_utc,text
0,Smart-Water-9833,1.737481e+09,There's a poll over at r/Baseball and a lot pe...
1,southshorerefugee,1.737686e+09,So the post banning X is bigger than the Yanke...
2,pumaunleashed,1.737565e+09,This thread is hilarious!!!!
3,twobridges94,1.737481e+09,Yogi Berra and Joe DiMaggio served in the war ...
4,AgathorKahn,1.737668e+09,I think it's very sad that this is the top pos...
...,...,...,...
3861,Neckwrecker,1.624584e+09,*eluded
3862,BenHogan1971,1.624589e+09,"this could have been boiled down to: ""Sanchez ..."
3863,samthewisetarly,1.624592e+09,"Sir, this is a Wendy's."
3864,[deleted],1.624603e+09,Flawed because Jlo don’t really got ass


In [ ]:
sox_posts = []

with open('redsox.json', 'r') as f:
	lines = f.readlines()

for line in lines:
	data_obj = json.loads(line)
	dobj = get_post(data_obj)
	sox_posts.append(dobj)
	comments = get_comment(data_obj)
	sox_posts.extend(comments)

sox_df = pd.DataFrame(sox_posts)
sox_df


,author,created_utc,text
0,WoodsmallConnor,1.540783e+09,Great season for the Red Sox that ends with th...
1,rusty378,1.540785e+09,MACHADO FELL TO HIS FUCKING KNEES FOR THE LAST...
2,sonicwonder,1.540785e+09,"Took out two 100+ win teams, then the #1 seed ..."
3,OSUCOWBOY1129,1.540784e+09,Man I love all you guys. This has been such an...
4,0DegreesCalvin,1.540786e+09,"Posted this in /r/baseball, but I’d like to re..."
...,...,...,...
1304,PhoenixUNI,1.506000e+09,D A N K\n\nA\n\nN\n\nK
1305,Blackcat008,1.506003e+09,Didn't Pedro get his in 1999?
1306,TylerW_511,1.506014e+09,does this really have ~1000 points or am I goi...
1307,Gordon_Goosegonorth,1.506025e+09,Curt Schilling is an idiot.


In [48]:
rays_posts = []

with open('tampabayrays.json', 'r') as f:
	lines = f.readlines()

for line in lines:
	data_obj = json.loads(line)
	dobj = get_post(data_obj)
	rays_posts.append(dobj)
	comments = get_comment(data_obj)
	rays_posts.extend(comments)

rays_df = pd.DataFrame(rays_posts)
rays_df


,author,created_utc,text
0,Bakerfuckingmayfield,1.730347e+09,SO DOES GLASNOW FTY
1,yungf69,1.730351e+09,KK gets to retire a World Series champion.
2,CollectionAmazing613,1.730347e+09,"I think Honeywell too, right."
3,Then_Ship1329,1.730347e+09,I have to tread lightly with what I’m about to...
4,grandmoffpoobah,1.730348e+09,"I'm so happy for him, the best possible end fo..."
...,...,...,...
927,OB4032,1.657238e+09,Lol Choi was eating an ice cream cone .
928,qawsedrf12,1.657237e+09,Waaaaaaayyyyyyy back when...\n\nJohnny Damon s...
929,catcherben27,1.657287e+09,Hey! That was me talking to Brett about the fa...
930,skylined45,1.657290e+09,Come to the light side.


In [46]:
jays_posts = []

with open('torontobluejays.json', 'r') as f:
	lines = f.readlines()

for line in lines:
	data_obj = json.loads(line)
	dobj = get_post(data_obj)
	jays_posts.append(dobj)
	comments = get_comment(data_obj)
	jays_posts.extend(comments)

jays_df = pd.DataFrame(jays_posts)
jays_df


,author,created_utc,text
0,batman2001,1.492223e+09,"We have hit 800, pls post aquarium related thi..."
1,SterlingAdmiral,1.492224e+09,I'm going to the game tomorrow but I'm going t...
2,patderp,1.492224e+09,I'm an Orioles fan but I must admit you guys r...
3,smoshr,1.492223e+09,*GIVE ME THE CUTTER GOOD DOCTOR*
4,[deleted],1.492223e+09,[when i visited Toronto last November](http://...
...,...,...,...
3120,admiral_bringdown,1.499054e+09,I feel like I'm taking fucking crazy pills. I'...
3121,thestratomaster1227,1.499059e+09,"Anyone else here high,and thought it was gonna..."
3122,[deleted],1.499084e+09,It's crazy.\nAt the end of the season the thou...
3123,[deleted],1.499094e+09,"Welp, never thought I'd see a headline like th..."
